## Chapter 17: modules and tests

This chapter covers both modules and tests.  Modules are another great way to separate code and put types and common functions together in that they will be used.

#### 17.1: The Revise package

The `Revise` package is a great package to run to build packages.  Recall that since `struct`s are immutable, if you are trying to design a type, then you can't change it once you've made it.  This made building a type and iterating on it difficult.  The `Revise` package handles this for you.  

In [2]:
using Revise

#### 17.2: Creating a module

A `module` is a block of code that often contains:

* custom types (structs)
* functions
* other data needed by the functions and types. 

A great example is that of the Playing Cards that we've been using in a few chapters in this course.  To creating, here's a template:

In [2]:
module PlayingCards

end

Main.PlayingCards

We can then put all of the related playing card types and functions inside this module. 

Instead of building this module in a jupyter cell, we'll often build it in a separate file.  This has already been done if you go to `../julia-files/PlayingCards.jl`, where `..` means up a directory.  You can open this inside jupyter. 

Then to load it, we will use the function `includet`, which is part of the `Revise` module.  The `t` stands for tracking in that the function `includet` will track changes to the module and reload as needed.

In [3]:
includet("../julia-files/PlayingCards.jl")

And since it is a module, we need to give access to it with the `using` command.  Note: the . in front of the name is because it is a local module

In [4]:
using .PlayingCards

Now we have access to all of the types and functions.

In [5]:
h = Hand("2♡,6♣,2♠,2♢,6♢")

[2♡,6♣,2♠,2♢,6♢]

In [6]:
isFullHouse(h)

true

If you notice, the module doesn't include the `runTrials` function we wrote in Chapter 15, so let's add that to the module.  (and don't forget to export it)

In [7]:
runTrials(isFullHouse,1_000_000)

LoadError: UndefVarError: runTrials not defined

#### Documenting functions in modules

In [8]:
?Card

search: Card ischardev PlayingCards CapturedException clipboard CartesianIndex



```
Card(r::Int, s::Int)
```

Create a Card object that represents a playing card with rank `r` and suit `s`.  The rank must satisfy `1<=r<=13` and the suit represents `1<=s<=4`.   In addition, one can make a Card with a single integer `n` that satifies `1<=n<=52`. Lastly, You can create a Card with a string consisting of the rank as `A,1,2,3,...,9,T,J,Q,K` and the suit ♣,♠,♡,♢. 

# Examples

```julia-repl
julia> Card(10,3)
T♡

julia> Card(33)
7♢

julia> Card("J♠")
J♠
```


In [9]:
?isFullHouse

search: isFullHouse



```
isFullHouse(h::Hand)
```

Returns a boolean if a given hand, `h` is a full house hand. 


#### 17.3: Unit Tests

A unit test is a way to ensure that code is written in a robust manner.  A test is a piece of code that will determine if something is returned true. In Julia, we will use the `Test` package

In [9]:
using Test

A very simple example is if we define a variable

In [10]:
n=3

3

and then test if this is 3 using the `@test` macro.

In [11]:
@test n==3

Test Passed

We'll start by testing some of the `PlayingCards` types and function.  First, we can test if create a `Card` actually makes a card.

In [12]:
@test isa(Card(1,4),Card)

Test Passed

which says yes, running the code `Card(1,4)` creates a Card object.  We can also test the other constructors as well:

In [13]:
@test isa(Card(35),Card)

Test Passed

In [14]:
@test isa(Card("3♣"),Card)

Test Passed

It's a good idea to also make sure that the checking routines work:

In [15]:
@test isa(Card(13,5),Card)

Error During Test at In[15]:1
  Test threw exception
  Expression: Card(13, 5) isa Card
  ArgumentError: The suit must be an integer between 1 and 4.
  Stacktrace:
   [1] Card(::Int64, ::Int64) at /Users/pstaab/code/sci-comp-notebooks/julia-files/PlayingCards.jl:17
   [2] top-level scope at In[15]:1
   [3] include_string(::Function, ::Module, ::String, ::String) at ./loading.jl:1091
   [4] execute_code(::String, ::String) at /Users/pstaab/.julia/packages/IJulia/a1SNk/src/execute_request.jl:27
   [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/pstaab/.julia/packages/IJulia/a1SNk/src/execute_request.jl:86
   [6] #invokelatest#1 at ./essentials.jl:710 [inlined]
   [7] invokelatest at ./essentials.jl:709 [inlined]
   [8] eventloop(::ZMQ.Socket) at /Users/pstaab/.julia/packages/IJulia/a1SNk/src/eventloop.jl:8
   [9] (::IJulia.var"#15#18")() at ./task.jl:356
  


LoadError: There was an error during testing

However, notice that there's an error that the test wasn't passed as well as the error that is throw from the constructor.  A better way to do this is to test if an error is thrown:

In [16]:
@test_throws ArgumentError Card(13,5)

Test Passed
      Thrown: ArgumentError

##### Test Sets

A nice way to collect tests together in common ways is to make a test set.  The following covers all successfully constructed Cards:

In [17]:
@testset "Legal Card Constructor" begin
        @test isa(Card(1,3),Card)
        @test isa(Card(45),Card)
        @test isa(Card("3\u2660"),Card)
        @test isa(Card("T♣"),Card)
    end

Test Summary:          | Pass  Total
Legal Card Constructor |    4      4


Test.DefaultTestSet("Legal Card Constructor", Any[], 4, false)

The idea of a test suite (many test sets that are associated with a module) is to have it on hand so when you update things, you don't break anything (have a regression). So we will put this in a file that we can just load

In [18]:
include("../julia-files/test-playing-cards.jl")

Test Summary:          | Pass  Total
Legal Card Constructor |    4      4
Test Summary:                   | Pass  Total
Illegal Cards throws exceptions |    5      5
Test Summary:          | Pass  Total
Legal Hand Constructor |    3      3
Test Summary:                  | Pass  Total
Illegal Hand throws exceptions |    5      5
Test Summary: | Pass  Total
Card Tests    |    3      3
Test Summary: | Pass  Total
Full House    |    2      2


Test.DefaultTestSet("Full House", Any[], 2, false)

#### 17.8: A Rootfinding Module

We will also create a Rootfinding module that has some of the functions that we have seen over this course.  They are in the `Rootfinding.jl` file in the `julia-files` directory:

In [3]:
includet("../julia-files/Rootfinding.jl")
using .Rootfinding

Take a look at that file:

In [4]:
newton(x->x^2-2,1)

The root is approximately x̂ = 1.4142135623746899
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.510614104447086e-12
which took 4 steps

In [5]:
newton(x->x^2+3,1, max_steps=19)

The root was not found within 19 steps.
Currently, the root is approximately x̂ = -1.0 
An estimate for the error is -2.0
with f(x̂) = 4.0


In [6]:
x = newton(x->x^2-2,1)

The root is approximately x̂ = 1.4142135623746899
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.510614104447086e-12
which took 4 steps

It would be good to include some tests with this module.  Let's test if the root above is actually $\sqrt{2}$.

In [9]:
@test x.root == sqrt(2)

Test Failed at In[9]:1
  Expression: x.root == sqrt(2)
   Evaluated: 1.4142135623746899 == 1.4142135623730951


LoadError: There was an error during testing

And this fails because the two are not exactly equal.  Recall that floating point numbers are equal only all bits are equal.  Instead we can test

In [10]:
@test abs(x.root-sqrt(2)) < 1e-6

Test Passed

In [11]:
x = newton(x->x^2+1,2)

The root was not found within 10 steps.
Currently, the root is approximately x̂ = 2.4008803928468465 
An estimate for the error is 1.4086971347905715
with f(x̂) = 6.764226660756428


In [12]:
@test !x.converged

Test Passed

In [13]:
@testset "function with no root" begin
  val = newton(x->x^2+1,2)
  @test !val.converged
  @test val.num_steps == val.max_steps
 end;

Test Summary:         | Pass  Total
function with no root |    2      2
